# Computational Graphs

In this notebook, some simple computational graph (CG) nodes are implemented. Each with a forward pass method (prop_forward) and a backward pass method (prop_backward). No assumption of being in a neural network is present, so the gradient is calculated considering all values to be instances of a variable of interest, this is, there is no concept or weights or data points. There are just inputs and outputs. The reason for this approach is that I consider it important to learn about computational graphs in order to understand neural networks better. After all, neural networks are not anything but that: ~~snowflaked~~ computational graphs

The assumption above does not impede nor complicate, however, the construction of a neural network, because a wrapper class (*e.g.* a Perceptron class) can use these CG components and be aware of weights and data. Then, such wrapper classes can be grouped in different ways to create more complex neural structures.

In [1]:
from abc import ABC, abstractmethod
import numpy as np

In [14]:
class CGNode(ABC):
    
    '''
    Computational graph node template. In all of the methods,
    the input vectors must have a shape of the form (n,)
    '''
    
    def __init__(self):
        self.last_input = None
        self.last_output = None
    
    @abstractmethod
    def prop_forward(self):
        
        '''
        Take inputs and evaluate the
        node's corresponding operation
        '''

        pass
    
    @abstractmethod
    def prop_backward(self, grad):
        
        '''
        Evaluate the gradient of the node's
        corresponding operation using an 
        incoming gradient 'grad' in backprop
        '''
        
        pass

In [3]:
class CGSum(CGNode):
    
    '''
    Computational graph sum node
    '''
    
    def prop_forward(self, x):
        self.last_input = np.array(x)
        self.last_output = np.sum(x)
        return self.last_output
    
    def prop_backward(self, grad):
        return grad * np.ones(self.last_input.size)


In [4]:
s = CGSum()
s.prop_forward([1,7,9])
s.prop_backward(1.2)

array([1.2, 1.2, 1.2])

In [5]:
class CGMul(CGNode):
    
    '''
    Computational graph multiplication node
    '''
    
    def prop_forward(self, x):
        self.last_input = np.array(x)
        self.last_output = np.prod(self.last_input)
        return self.last_output
    
    def prop_backward(self, grad):        
        # repeat x size times and arrange as rows
        
        size = self.last_input.size
        
        repeated = np.ones((size, size))*self.last_input
        
        # start differentiation
        np.fill_diagonal(repeated, 1)
        
        # multiply constants 
        # (variables not being differentiated)
        return grad * np.prod(repeated, 1)

In [6]:
a = CGMul()
a.prop_forward([1,2,3,8])

48

In [12]:
a.prop_backward(1)

array([48., 24., 16.,  6.])

In [8]:
class CGExp(CGNode):
    
    def prop_forward(self, x):
        self.last_input = np.array(x)
        self.last_output = np.exp(self.last_input)
        return self.last_output
    
    def prop_backward(self, grad):
        return grad * self.last_output

In [9]:
class CGDot:
    
    '''
    Dot product node composed of summation and multiplication nodes for 
    transparency and accordance with the concept of a computational graph
    '''
    
    def __init__(self):
        self.last_x = None
        self.last_y = None
        self.last_output = None
        
        # one summation
        self.cg_sum = None
        
        # many multiplications
        self.cg_muls = None
    
    def prop_forward(self, x, y):
        
        '''Compute the dot product of vectors x and y'''
        
        self.last_x = np.array(x)
        self.last_y = np.array(y)
        
        # if no data has been propagated yet,
        # instantiate as many mul nodes
        # as the dimension of the inputs
        if not self.cg_sum:
            self.cg_muls = [CGMul() for _ in self.last_x]
            self.cg_sum = CGSum()
        
        # align corresponding values to multiply
        xy = CGDot.pre_dot(x, y)
        
        # multiplication phase
        summands = [cg_mul.prop_forward(factors) for cg_mul, factors in zip(self.cg_muls, xy)]
        
        # summation
        self.last_output = self.cg_sum.prop_forward(summands)
        
        return self.last_output
    
    def prop_backward(self, grad, flat=False):
        
        '''
        Propagate the gradients backward by calling prop_backward
        of the sum and multiplications of which the dot product is 
        composed of. Actual computation is delegated to smaller 
        nodes
        
        :returns: nx2 array (n being the size of the input). Each
        row is the gradient of a sum of two input vector components
        unless flat is True, in which case the return value is just
        the flattened version of the nx2 array
        '''
        
        sum_grad = self.cg_sum.prop_backward(grad)
        mul_grad = np.array([
            cg_mul.prop_backward(sum_grad_i) for cg_mul, sum_grad_i in zip(self.cg_muls, sum_grad)
        ])
        
        return mul_grad.flatten() if flat else mul_grad
    
    
    @staticmethod
    def pre_dot(x, y):
        
        '''
        Arrange the inputs to facilitate 
        dot product computation
        '''
        
        x = np.reshape(x, (1, len(x)))
        y = np.reshape(y, (1, len(y)))
        xy = np.concatenate((x,y))
        return xy.T

#### CGDot in action

In [10]:
dot = CGDot()
dot.prop_forward([1,2,3], [4,5,6])

32

In [11]:
dot.prop_backward(grad=1)

array([[4., 1.],
       [5., 2.],
       [6., 3.]])